# Setup

## Packages

In [1]:
%run /home/ubuntu/work/therapeutic_accelerator/scripts/base.py

In [2]:
import pandas as pd
import numpy as np

from transformers import T5Tokenizer # AutoModel, AutoTokenizer, BertTokenizer,BioGptModel, BioGptConfig, BioGptTokenizer
import torch

## Tokenizers and Models

In [2]:
max_sequence_length = 1200
embedding_size = 200
T5tokens = T5Tokenizer.from_pretrained('t5-base', model_max_length = max_sequence_length)

In [ ]:
# bio_bert_model = AutoModel.from_pretrained("gsarti/biobert-nli")
# bio_bert_tokenizer = AutoTokenizer.from_pretrained("gsarti/biobert-nli")
# original_bert_tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
# T5Abstract_model = TFT5ForConditionalGeneration.from_pretrained('t5-base')
# biogpttokenizer = BioGptTokenizer.from_pretrained("microsoft/biogpt")
# biogptmodel = BioGptModel.from_pretrained("microsoft/biogpt")

Custom embeddings function

In [ ]:
from chromadb.api.types import Documents, EmbeddingFunction, Embeddings

class MyEmbeddingFunction(EmbeddingFunction):
    def __call__(self, texts: Documents) -> Embeddings:
        #create document embeddings with T5
        embeddings = []
        for text in texts:
            input_ids = T5tokens.encode(text)
            input_ids = torch.tensor(input_ids).unsqueeze(0)
            with torch.no_grad():
                output = T5Abstract_model(input_ids)
            embeddings.append(output[0][0][0].numpy())
        return embeddings

# Langchain Sentence Embeddings

In [ ]:
from abc import ABC
from typing import List, Optional, Any

import chromadb
from langchain.docstore.document import Document
from langchain.embeddings.base import Embeddings
from langchain.vectorstores import Chroma
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.embeddings import OpenAIEmbeddings

In [3]:
embeddings_model = OpenAIEmbeddings(openai_api_key=keys['openai_api_key'])

In [ ]:
embeddings = OpenAIEmbeddings()
vectorstore = CachedChroma.from_documents_with_cache(
    ".persisted_data", texts, embeddings, collection_name="fun_experiement"
)

In [ ]:
# create a function that uses langchain to embed sentences
def embed_sentences(sentences, tokenizer, model, max_sequence_length):
    # tokenize the sentences
    tokenized_sentences = tokenizer(sentences, padding=True, truncation=True, return_tensors="pt", max_length=max_sequence_length)
    # get the embeddings
    with torch.no_grad():
        model_output = model(**tokenized_sentences)
    # get the embeddings from the model output
    embeddings = model_output[0][:,0,:].numpy()
    return embeddings

# Create Encodings using Dask

## Full Text

In [54]:
# import dask
import sqlalchemy as sa
from dask import dataframe as dd
from dask.delayed import delayed
from dask.diagnostics import ProgressBar
import json
# from glob import glob

In [51]:
table_name = 'fulltext'

In [193]:
def create_dictionary(text):
    """ Turn string containing list of dictionaries into a dictionary"""
    
    # remove new line characters
    categories = re.sub(r'[\[\]\'\\]', '', text)

    # remove outer brackets, quotes, and split on commas
    categories = categories.strip('{}').strip('"').split('","')

    # create list with unique values from category
    # categories = pd.Series([json.loads(t)['category'] for t in categories]).unique().tolist()
    categories = [json.loads(t) for t in categories]
    
    return categories

In [223]:
# Get attributes table for the metadata embeddings
sql = sa.text(f''' 
    SELECT * from attributes limit 100;
''')

with engine.connect() as conn: 
    query = conn.execute(sql)
    
att = pd.DataFrame(query.fetchall())


In [224]:
att

,index,corpusid,externalids,url,title,authors,venue,publicationvenueid,year,referencecount,citationcount,influentialcitationcount,isopenaccess,s2fieldsofstudy,publicationtypes,publicationdate,journal,updated,id
0,1428357,209166261,"{""ACL"": null, ""DBLP"": null, ""ArXiv"": null, ""MA...",https://www.semanticscholar.org/paper/786dd3d7...,Characterising risk of homicide in a populatio...,"{""{\""authorId\"": \""1435073478\"", \""name\"": \""M...",Journal of Epidemiology and Community Health,27475f31-a1d2-401b-84ad-9b405c7609a8,2019.0,52,5,0,True,"{""{\""category\"": \""Medicine\"", \""source\"": \""s...",{JournalArticle},2019-12-09,"{""name"": ""Journal of Epidemiology & Community ...",2022-12-13T06:49:16.062Z,4201762
1,1428358,34851144,"{""ACL"": null, ""DBLP"": null, ""ArXiv"": null, ""MA...",https://www.semanticscholar.org/paper/0ee68535...,The complete primary structure of the marine c...,"{""{\""authorId\"": \""46836751\"", \""name\"": \""M. ...",Journal of Protein Chemistry,8f570d52-36ae-4027-b110-e71f75f80727,1991.0,46,4,0,True,"{""{\""category\"": \""Biology\"", \""source\"": \""s2...","{JournalArticle,Study}",1991-06-01,"{""name"": ""Journal of Protein Chemistry"", ""page...",2022-12-16T15:39:12.451Z,4201772
2,1428359,81389011,"{""ACL"": null, ""DBLP"": null, ""ArXiv"": null, ""MA...",https://www.semanticscholar.org/paper/f825e998...,Mind over chatter: plastic up-regulation of th...,"{""{\""authorId\"": \""2725179\"", \""name\"": \""R. L...",None,None,2012.0,69,3,0,True,"{""{\""category\"": \""Psychology\"", \""source\"": \...",None,2012-04-02,"{""name"": ""Nature Precedings"", ""pages"": ""1-1"", ...",2022-12-19T23:01:44.020Z,4201800
3,1428360,58896665,"{""ACL"": null, ""DBLP"": null, ""ArXiv"": null, ""MA...",https://www.semanticscholar.org/paper/343f1ff5...,Evaluation of a novel luciferase reporter cons...,"{""{\""authorId\"": \""5160468\"", \""name\"": \""T. T...",None,None,2008.0,20,6,0,False,"{""{\""category\"": \""Biology\"", \""source\"": \""s2...",None,None,"{""name"": ""African Journal of Biotechnology"", ""...",2023-02-15T08:29:34.881Z,4201820
4,1428361,95710584,"{""ACL"": null, ""DBLP"": null, ""ArXiv"": null, ""MA...",https://www.semanticscholar.org/paper/035f56f4...,Die Chloro-rhodate(III) [RhCl6]3- und [RhCl5(H...,"{""{\""authorId\"": \""21586292\"", \""name\"": \""U. ...",None,None,1986.0,2,7,0,True,"{""{\""category\"": \""Chemistry\"", \""source\"": \""...",None,None,"{""name"": ""Zeitschrift f\u00fcr Naturforschung ...",2023-02-16T08:00:04.091Z,4201827
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,1428461,231743747,"{""ACL"": null, ""DBLP"": null, ""ArXiv"": null, ""MA...",https://www.semanticscholar.org/paper/b864a710...,Repair of the Posterior Lateral Meniscal Root ...,"{""{\""authorId\"": \""6742595\"", \""name\"": \""M. L...",Arthroscopy Techniques,dc86599c-b4fe-4532-973d-f3785213b912,2021.0,25,5,0,False,"{""{\""category\"": \""Medicine\"", \""source\"": \""s...",{JournalArticle},2021-01-01,"{""name"": ""Arthroscopy Techniques"", ""pages"": ""e...",2023-05-02T13:37:28.531Z,4216004
96,1428462,13931135,"{""ACL"": null, ""DBLP"": null, ""ArXiv"": null, ""MA...",https://www.semanticscholar.org/paper/23c7b684...,Genetic Mutation Analysis of Parkinson’s Disea...,"{""{\""authorId\"": \""90411938\"", \""name\"": \""A. ...",Molecular Diagnosis & Therapy,2f4137af-1a75-458e-95e8-bc1db564ffc8,2016.0,28,17,1,False,"{""{\""category\"": \""Biology\"", \""source\"": \""s2...",{JournalArticle},2016-06-13,"{""name"": ""Molecular Diagnosis & Therapy"", ""pag...",2023-05-06T15:53:06.058Z,4216018
97,1428463,143521233,"{""ACL"": null, ""DBLP"": null, ""ArXiv"": null, ""MA...",https://www.semanticscholar.org/paper/9c01a887...,Perceiving luxury and necessity,"{""{\""authorId\"": \""145329337\"", \""name\"": \""S....",None,None,1998.0,15,191,10,False,"{""{\""category\"": \""Psychology\"", \""source\"": \...",None,1998-10-01,"{""name"": ""Journal of Economic Psychology"", ""pa...",2023-

In [ ]:
# # turn strings into list of dictionaries
# att['s2fieldsofstudy'] = att['s2fieldsofstudy'].apply(create_dictionary).apply(lambda x: pd.Series([d['category'] for d in x]).unique().tolist())
# att['authors'] = att['authors'].apply(create_dictionary)

# # Keep relevant columns
# att.drop(axis = 1, columns=['index', 'url', 'title', 'publicationvenueid', 'updated'], inplace=True)

In [225]:
sql = sa.text(f''' 
    SELECT * FROM {table_name} LEFT JOIN attributes ON ({table_name}.corpusid = CAST(attributes.corpusid as text)) LIMIT 10;
''')

with engine.connect() as conn: 
    query = conn.execute(sql)
    
ft = pd.DataFrame(query.fetchall())

In [ ]:
# Get locations of figure captions to remvoe from full text
ft['annotations.figurecaption'] = ft['annotations.figurecaption'].apply(json.loads)

In [226]:
# turn strings into list of dictionaries
ft['s2fieldsofstudy'] = ft['s2fieldsofstudy'].apply(create_dictionary).apply(lambda x: pd.Series([d['category'] for d in x]).unique().tolist())
ft['authors'] = ft['authors'].apply(create_dictionary)


In [228]:
ft.columns.tolist()

['empty',
 'Unnamed: 0',
 'corpusid',
 'text',
 'source.pdfurls',
 'source.pdfsha',
 'source.oainfo',
 'annotations.abstract',
 'annotations.author',
 'annotations.authoraffiliation',
 'annotations.authorfirstname',
 'annotations.authorlastname',
 'annotations.bibauthor',
 'annotations.bibauthorfirstname',
 'annotations.bibauthorlastname',
 'annotations.bibentry',
 'annotations.bibref',
 'annotations.bibtitle',
 'annotations.bibvenue',
 'annotations.figure',
 'annotations.figurecaption',
 'annotations.figureref',
 'annotations.formula',
 'annotations.paragraph',
 'annotations.publisher',
 'annotations.sectionheader',
 'annotations.table',
 'annotations.tableref',
 'annotations.title',
 'annotations.venue',
 'source.oainfo.license',
 'source.oainfo.openaccessurl',
 'source.oainfo.status',
 'id',
 'index',
 'corpusid',
 'externalids',
 'url',
 'title',
 'authors',
 'venue',
 'publicationvenueid',
 'year',
 'referencecount',
 'citationcount',
 'influentialcitationcount',
 'isopenaccess',


In [ ]:
def get_text_from_index(text, indexes): 
    """ index is a list of dictionary with start and end keys"""
    # Looking at sections
    section = {}
    for i in indexes: 
        section['name'] = text['text'][i['start']:i['end']]
        section['start'] = i['start']
        section['end'] = i['end']
        
    return section

In [210]:
# Keep relevant columns
ft = ft[
    [
        "text",
        "corpusid",
        "title",
        "s2fieldsofstudy",
        "authors",
        "venue",
        "year",
        "referencecount",
        "citationcount",
        "influentialcitationcount",
        "isopenaccess",
        "s2fieldsofstudy",
        "publicationtypes",
        "publicationdate",
        "journal",
    ]
]

In [35]:
from langchain.text_splitter import CharacterTextSplitter

text_splitter = CharacterTextSplitter(
    separator = "\n\n",
    chunk_size = 2000,
    chunk_overlap  = 200,
    length_function = len,
)

In [36]:
# from langchain.document_loaders.csv_loader import CSVLoader
# loader = CSVLoader(file_path='./example_data/mlb_teams_2012.csv')
# data = loader.load()

In [220]:
# split the text into chunks
documents = text_splitter.create_documents([ft.loc[0, 'text']])

# create metadata dictionary
metadata = ft.iloc[0, 1:].to_dict()

for i, d in enumerate(documents): 
    d.metadata = metadata | {'chunk_id': i}
    
documents

Created a chunk of size 2667, which is longer than the specified 2000


[Document(page_content='Biologic Therapy Carries a Very Low Risk of Reactivation in Hepatitis B Surface Antigen-Negative Phase of Hepatitis B\n\n\nİlkay Ergenç ergencilkay@gmail.com \nDivision of Gastroenterology\nDepartment of Internal Medicine\nFaculty of Medicine\nMarmara University\nİstanbulTurkey\n\nHaluk Tarık Kani \nDivision of Gastroenterology\nDepartment of Internal Medicine\nFaculty of Medicine\nMarmara University\nİstanbulTurkey\n\nMurat Karabacak \nDivision of Rheumatology\nDepartment of Internal Medicine\nFaculty of Medicine\nMarmara University\nİstanbulTurkey\n\nElif Cömert Özer \nDepartment of Dermatology\nFaculty of Medicine\nMarmara University\nİstanbulTurkey\n\nShahin Mehdiyev \nDivision of Gastroenterology\nDepartment of Internal Medicine\nFaculty of Medicine\nMarmara University\nİstanbulTurkey\n\nFuad Jafarov \nDivision of Gastroenterology\nDepartment of Internal Medicine\nFaculty of Medicine\nMarmara University\nİstanbulTurkey\n\nKerem Yiğit Abacar \nDivision of Rh

In [28]:
import re

In [229]:
# create me a function that will preprocess text to prepare to be used in a nlp model
import unidecode

def remove_accented_chars(text):
    """remove accented characters from text, e.g. café"""
    text = unidecode.unidecode(text)
    return text

def preprocess_text(text):
    # remove the new lines
    text = text.replace('\n', ' ')
    # remove multiple spaces
    text = re.sub(r'\s+', ' ', text)
    
    text = remove_accented_chars(text)
    return text

ModuleNotFoundError: No module named 'unidecode'

In [222]:
preprocess_text(documents[0].page_content)

'Biologic Therapy Carries a Very Low Risk of Reactivation in Hepatitis B Surface Antigen-Negative Phase of Hepatitis B İlkay Ergenç ergencilkay@gmail.com Division of Gastroenterology Department of Internal Medicine Faculty of Medicine Marmara University İstanbulTurkey Haluk Tarık Kani Division of Gastroenterology Department of Internal Medicine Faculty of Medicine Marmara University İstanbulTurkey Murat Karabacak Division of Rheumatology Department of Internal Medicine Faculty of Medicine Marmara University İstanbulTurkey Elif Cömert Özer Department of Dermatology Faculty of Medicine Marmara University İstanbulTurkey Shahin Mehdiyev Division of Gastroenterology Department of Internal Medicine Faculty of Medicine Marmara University İstanbulTurkey Fuad Jafarov Division of Gastroenterology Department of Internal Medicine Faculty of Medicine Marmara University İstanbulTurkey Kerem Yiğit Abacar Division of Rheumatology Department of Internal Medicine Faculty of Medicine Marmara University İ

In [21]:
text_splitter.split_text(ft.loc[0, 'text'])

['OPEN ACCESS EDITED BY\n\n\nAlexander Nikolaevich Orekhov \nGeorgy Guria \nAnton G Kutikhin \nIvan Melnikov \nZufar Gabbasov zufargabbasov@yandex.ru \nMelnikov I \nKozlov S \nPogorelova O \nTripoten M \nKhamchieva L \nSaburova O \nAvtaeva Y \nZvereva M \nMatroze E \nKuznetsova T \nProkofieva L \nBalakhonova T \nGabbasov Z \n\nInstitute for Aterosclerosis Research\nRussian Academy of Medical Sciences\nNational Research Centre for Haematology\nRussia REVIEWED BY, Russia, Russia\n\n\nCORRESPONDENCE\nSPECIALTY SECTION\nDepartment of Ultrasound Diagnostics\nLaboratory of Cell Hemostasis, National Medical Research Centre of Cardiology named after academician E.I. Chazov of the Ministry of Health of the Russian Federation, Moscow, Russia, Laboratory of Gas Exchange, Biomechanics and Barophysiology, State Scientific Center of the Russian Federation -The Institute of Biomedical Problems of the Russian Academy of Sciences, Moscow, Russia, Laboratory of Problems of Atherosclerosis, National Medi

In [16]:
metadatas = ... # from attributes table in postgres

[Document(page_content='OPEN ACCESS EDITED BY\n\n\nAlexander Nikolaevich Orekhov \nGeorgy Guria \nAnton G Kutikhin \nIvan Melnikov \nZufar Gabbasov zufargabbasov@yandex.ru \nMelnikov I \nKozlov S \nPogorelova O \nTripoten M \nKhamchieva L \nSaburova O \nAvtaeva Y \nZvereva M \nMatroze E \nKuznetsova T \nProkofieva L \nBalakhonova T \nGabbasov Z \n\nInstitute for Aterosclerosis Research\nRussian Academy of Medical Sciences\nNational Research Centre for Haematology\nRussia REVIEWED BY, Russia, Russia\n\n\nCORRESPONDENCE\nSPECIALTY SECTION\nDepartment of Ultrasound Diagnostics\nLaboratory of Cell Hemostasis, National Medical Research Centre of Cardiology named after academician E.I. Chazov of the Ministry of Health of the Russian Federation, Moscow, Russia, Laboratory of Gas Exchange, Biomechanics and Barophysiology, State Scientific Center of the Russian Federation -The Institute of Biomedical Problems of the Russian Academy of Sciences, Moscow, Russia, Laboratory of Problems of Atherosc

In [ ]:
docs = [Document(page_content=t) for t in text[:3]]

In [ ]:
# ft = pd.read_sql_table("fulltext", engine.connect())

In [ ]:
# Create connection to postgres db
# from sqlalchemy.engine.url import URL

# postgres_db = {'drivername': 'postgres',
#                'database': 'postgres',
#                'username': 'postgres',
#                'password': keys["postgres"],
#                'host': config["database"]["host"],
#                'port': 5432}
# print(URL(**postgres_db))
# postgres = URL(**postgres_db)

In [4]:
df = pd.read_sql_table('fulltext', con = f'postgresql://postgres:{keys["postgres"]}@{config["database"]["host"]}:5432/postgres')

: 

: 

In [18]:
ddf = dd.read_sql_table('fulltext', 
                        con = f'postgresql://postgres:{keys["postgres"]}@{config["database"]["host"]}:5432/postgres',
                        index_col = 'id',
                        head_rows = 10,
                        npartitions = 100)

# Remove unnecessary columns
# ddf = ddf.drop(columns = ['index'])

# Remove empty abstract rows
# ddf = ddf.dropna(how = 'all', subset='abstract').reset_index(drop = True)

AttributeError: 'OptionEngine' object has no attribute 'execute'

In [ ]:

for x in xl_files:
    parts = dask.delayed(try_to_read)(x)
    # filter_df = dask.delayed(get_techniques)(parts)
    output.append(parts)

# convert to a single dataframe
df_total = dd.from_delayed(output)

# df_total.visualize()

with ProgressBar():
    ddf = df_total.compute()

# Upload to Postgresql DB

In [ ]:
# sql = text(''' 
#     SELECT EXISTS (
#         SELECT FROM information_schema.tables 
#         WHERE    table_name   = 'abstracts'
#     );
# ''')

# with engine.connect() as conn: 
#     conn.execute(sql)

Create array columns to store encoding and mask

In [ ]:
table_name = 'abstracts_encodings'

In [ ]:
delete_table = False

if delete_table: 
    sql = text(f''' 
        DROP TABLE IF EXISTS {table_name}};
    ''')

    with engine.connect() as conn: 
        query = conn.execute(sql)

In [ ]:
# create table
# Create Table in DB first before uploading
from sqlalchemy import MetaData, Table, Column, Integer, String, ARRAY

metadata_obj = MetaData()

# Create abstracts metadata
abstracts = Table(
    table_name,
    metadata_obj,
    Column("paperId", String, nullable = True),
    Column("corpusId", String, nullable=True),
    Column("abstract", String, nullable = True),
    Column("input_ds", ARRAY(Integer), nullable=True),
    Column("attention_mask", ARRAY(Integer), nullable=True),
)

metadata_obj.create_all(engine)

In [ ]:
# Upload dask dataframe to psql
ddf = ddf.to_sql(name = table_name, uri = str(url_object), if_exists = 'replace', index = False, chunksize = 10000, method = 'multi')

In [4]:
# Check if it worked
import pandas as pd
from sqlalchemy import text

table_name = 'abstracts_encodings'

sql = text(f''' 
    SELECT * FROM {table_name} LIMIT 5;
''')

with engine.connect() as conn: 
    query = conn.execute(sql)

test = pd.DataFrame(query.fetchall())
test.head()

,paperId,corpusId,abstract,input_ids,attention_mask
0,f674f1fa1bcccc7c4072aed1eca9cc3c6f773539,9445537,Introduction Meta-epidemiological studies exam...,"[18921, 14204, 18, 15, 102, 23, 1778, 23, 4478...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."
1,c0bbe6967f8c4c9223112bb9fe384f1af2d28339,216030749,Purpose: To assess whether treatment with the ...,"[7333, 2748, 15, 10, 304, 6570, 823, 1058, 28,...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."
2,4e112781c353ff25ea93a796f51e6a71ab7a52af,43026158,The aim of the study is to examine whether bas...,"[37, 2674, 13, 8, 810, 19, 12, 5443, 823, 2072...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."
3,d79b0987af87e240fcfd1d86a7ecf0b16638139f,231849048,The scale of the SARS-CoV-2 pandemic has thrus...,"[37, 2643, 13, 8, 180, 25210, 18, 3881, 553, 4...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."
4,6644258fd62e695b15ef6825099a8dd4e0710334,3701623,Group 2 innate lymphoid cells (ILC2s) are impo...,"[1531, 204, 3, 28538, 25049, 32, 23, 26, 2640,...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."


In [17]:
# Check if it worked
import pandas as pd
from sqlalchemy import text

table_name = 'fulltext'

sql = text(f''' 
    SELECT * FROM {table_name} LIMIT 5;
''')

with engine.connect() as conn: 
    query = conn.execute(sql)

test = pd.DataFrame(query.fetchall())
test.head()

,empty,Unnamed: 0,corpusid,text,source.pdfurls,source.pdfsha,source.oainfo,annotations.abstract,annotations.author,annotations.authoraffiliation,...,annotations.publisher,annotations.sectionheader,annotations.table,annotations.tableref,annotations.title,annotations.venue,source.oainfo.license,source.oainfo.openaccessurl,source.oainfo.status,id
0,1410,1410,250929149,\nOPEN ACCESS EDITED BY\n\n\nAlexander Nikolae...,None,57f2173016853ed6f6406a962adb11da43917165,None,None,"[{""end"":55,""start"":25},{""end"":69,""start"":56},{...","[{""end"":455,""start"":306},{""end"":1102,""start"":4...",...,None,"[{""end"":3001,""start"":2989},{""end"":6895,""start""...","[{""end"":35560,""start"":34552},{""end"":38099,""sta...","[{""end"":18195,""start"":18188},{""end"":18790,""sta...","[{""end"":22,""start"":1},{""end"":1721,""start"":1700}]",None,None,None,None,1
1,1411,1411,215793097,\n\n\n\nCi Song \nState Key Laboratory of Repr...,None,be45f9e9d6d1f4f713546b943cb2ff455989adc2,None,None,"[{""end"":228,""start"":4},{""end"":380,""start"":229}...","[{""end"":104,""start"":13},{""end"":227,""start"":106...",...,None,"[{""end"":8781,""start"":8771}]",None,"[{""end"":4906,""start"":4898}]",None,None,None,https://academic.oup.com/biolreprod/article-pd...,BRONZE,2
2,1412,1412,246020601,\nCOVID-19 and the Otolaryngology Residency Ma...,None,b7decbb79e55bb3c9b5430bef4323775813c39e0,None,"[{""end"":2210,""start"":391}]","[{""end"":109,""start"":85},{""end"":127,""start"":110...",None,...,None,"[{""end"":2224,""start"":2212},{""end"":4135,""start""...","[{""end"":16114,""start"":15573},{""end"":16650,""sta...","[{""attributes"":{""ref_id"":""tab_0""},""end"":7027,""...","[{""end"":82,""start"":1},{""end"":227,""start"":146}]",None,None,https://www.ncbi.nlm.nih.gov/pmc/articles/PMC9...,GREEN,3
3,1413,1413,18163935,\nAnalysis of biophysical and functional conse...,None,895fdf69460791863817c037332c0403c3e917b1,None,"[{""end"":1608,""start"":861}]","[{""end"":217,""start"":201},{""end"":235,""start"":21...","[{""end"":363,""start"":257},{""end"":410,""start"":366}]",...,None,"[{""end"":7790,""start"":7769},{""end"":7810,""start""...","[{""end"":32871,""start"":32129}]","[{""attributes"":{""ref_id"":""tab_0""},""end"":12397,...","[{""end"":95,""start"":1},{""end"":506,""start"":412}]",None,CCBY,https://doi.org/10.1002/1873-3468.12346,HYBRID,4
4,1414,1414,237935836,\nImpact of Plant-Based Meat Alternatives on t...,None,acf534c72039d2b2c10544b12f7cc872c2ba1b2a,None,None,"[{""end"":289,""start"":123},{""end"":375,""start"":29...","[{""end"":219,""start"":148},{""end"":288,""start"":22...",...,None,"[{""attributes"":{""n"":""1.""},""end"":1115,""start"":1...","[{""end"":60480,""start"":59841},{""end"":61332,""sta...","[{""attributes"":{""ref_id"":""tab_1""},""end"":8301,""...","[{""end"":95,""start"":1},{""end"":755,""start"":661}]",None,CCBY,https://www.mdpi.com/2304-8158/10/9/2040/pdf,GOLD,5


# Create Vectore Store

In [ ]:
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.vectorstores import Chroma
from langchain.text_splitter import CharacterTextSplitter
from langchain.llms import OpenAI
from langchain.chains import RetrievalQA

llm = OpenAI(temperature=0)